In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [3]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [4]:
train_data_dir = '/content/drive/MyDrive/Kaggle3/Train_Alphabet/'
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'

In [5]:
def load_and_preprocess_images(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith('.png'):
                    image_path = os.path.join(label_dir, filename)
                    # Check if the image is loaded successfully
                    image = cv2.imread(image_path)
                    if image is not None:  # Ensure image is not empty
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
                        image = cv2.resize(image, (224, 224))  # Resize image to (224, 224)
                        images.append(image)
                        labels.append(label)
                    else:
                        print(f"Warning: Unable to load image '{image_path}'")
    return np.array(images), np.array(labels)

In [6]:
# Step 3: Load and preprocess training and testing images
X_train, y_train = load_and_preprocess_images(train_data_dir)

In [7]:
np.save('X_train1_res.npy', X_train)
np.save('y_train1_res.npy', y_train)

In [ ]:
X_test = np.load('X_test1_vgg.npy')
y_test = np.load('y_test1_vgg.npy')

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Load ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Step 3: Add new layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Step 4: Compile the model
# We'll freeze the base model layers and only train the top layers initially
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 5: Train the model
history = model.fit(
    preprocess_input(X_train), y_train_encoded, epochs=10, batch_size=32, validation_data=(preprocess_input(X_test), y_test_encoded)
)

In [ ]:
# Step 6: Fine-tune the entire model
# Unfreeze the base model layers and recompile the model with a lower learning rate
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('resnet_image_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

In [ ]:
# Train the entire model again
history_fine_tune = model.fit(
    preprocess_input(X_train), y_train_encoded, epochs=10, batch_size=32, validation_data=(preprocess_input(X_test), y_test_encoded), callbacks=[checkpoint]
)

In [ ]:
# Step 5: Evaluate model
y_pred = lstm_model.predict(preprocess_input(X_test))
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_encoded, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
report = classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_)

print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Precision: {:.2f}%".format(precision * 100))
print("Recall: {:.2f}%".format(recall * 100))
print("F1 Score: {:.2f}%".format(f1 * 100))
print("Classification Report:\n", report)

In [ ]:
# Combine the histories from both training phases
history.history.update(history_fine_tune.history)

# Step 7: Plotting the loss and accuracy curves
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()